# 日本取引所グループに上場されている証券コードの取得

日本取引所グループに公開されている上場銘柄一覧情報（EXCELファイル）を加工して企業名から証券コードを取得するpythonコードを書いていきます。

---
[Google Colaboratoryで開く](https://colab.research.google.com/github/202408pythonciot/Data_Science_in_Finance/blob/main/Colaboratory/Base/get_jpx_ticker_list.ipynb)

<a target="_blank" href="https://colab.research.google.com/github/202408pythonciot/Data_Science_in_Finance/blob/main/Colaboratory/Base/get_jpx_ticker_list.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

---

## 1. 日本取引所グループからファイルをダウンロード
wgetコマンドを使い，日本取引所グループの上場銘柄一覧情報（EXCELファイル）をダウンロードします。


In [ ]:
!wget 'https://www.jpx.co.jp/markets/statistics-equities/misc/tvdivq0000001vg2-att/data_j.xls' -O data_j.xls

--2024-11-02 03:29:54--  https://www.jpx.co.jp/markets/statistics-equities/misc/tvdivq0000001vg2-att/data_j.xls
Resolving www.jpx.co.jp (www.jpx.co.jp)... 23.201.166.226
Connecting to www.jpx.co.jp (www.jpx.co.jp)|23.201.166.226|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 812032 (793K) [application/vnd.ms-excel]
Saving to: ‘data_j.xls’

data_j.xls          100%[===================>] 793.00K  2.62MB/s    in 0.3s    

2024-11-02 03:29:55 (2.62 MB/s) - ‘data_j.xls’ saved [812032/812032]



## 2. Pandasで読み込む
pandasを使って，Excelファイルを読み込みます。

In [ ]:
import pandas as pd

df = pd.read_excel('data_j.xls')
df

,日付,コード,銘柄名,市場・商品区分,33業種コード,33業種区分,17業種コード,17業種区分,規模コード,規模区分
0,20240930,1301,極洋,プライム（内国株式）,50,水産・農林業,1,食品,7,TOPIX Small 2
1,20240930,1305,ｉＦｒｅｅＥＴＦ ＴＯＰＩＸ（年１回決算型）,ETF・ETN,-,-,-,-,-,-
2,20240930,1306,ＮＥＸＴ ＦＵＮＤＳ ＴＯＰＩＸ連動型上場投信,ETF・ETN,-,-,-,-,-,-
3,20240930,1308,上場インデックスファンドＴＯＰＩＸ,ETF・ETN,-,-,-,-,-,-
4,20240930,1309,ＮＥＸＴ ＦＵＮＤＳ ＣｈｉｎａＡＭＣ・中国株式・上証５０連動型上場投信,ETF・ETN,-,-,-,-,-,-
...,...,...,...,...,...,...,...,...,...,...
4383,20240930,9991,ジェコス,プライム（内国株式）,6050,卸売業,13,商社・卸売,7,TOPIX Small 2
4384,20240930,9993,ヤマザワ,スタンダード（内国株式）,6100,小売業,14,小売,7,TOPIX Small 2
4385,20240930,9994,やまや,スタンダード（内国株式）,6100,小売業,14,小売,7,TOPIX Small 2
4386,20240930,9996,サトー商会,スタンダード（内国株式）,6050,卸売業,13,商社・卸売,-,-


## 3. 市場・商品区分による分類

In [ ]:
PrimeCodeList = df.loc[df["市場・商品区分"] == "プライム（内国株式）"]
StandardCodeList = df.loc[df["市場・商品区分"] == "グロース（内国株式）"]
GrowthCodeList = df.loc[df["市場・商品区分"] == "スタンダード（内国株式）"]
ETFCodeList = df.loc[df["市場・商品区分"] == "ETF・ETN"]

試しにプライムのコードと銘柄一覧を表示してみます。


In [ ]:
PrimeCodeList[["コード", "銘柄名"]]

,コード,銘柄名
0,1301,極洋
19,1332,ニッスイ
20,1333,マルハニチロ
40,1375,雪国まいたけ
42,1377,サカタのタネ
...,...,...
4380,9987,スズケン
4381,9989,サンドラッグ
4382,9990,サックスバー ホールディングス
4383,9991,ジェコス


## 4. Pythonの辞書(dict)化
企業名から証券コードを取得できるように，銘柄名をキーに，証券コードを値としてpythonの辞書コードにします。

In [ ]:
PrimeCode = dict(zip(PrimeCodeList["銘柄名"], PrimeCodeList["コード"]))

# テストとして「雪国まいたけ」の証券コードを取得
print(PrimeCode['雪国まいたけ'])

<class 'int'>


## ※ yfinanceでの使用
Pythonから金融情報を取得するためのライブラリであるyfinanceでは，日本取引所グループで扱われている証券コードに".T"がついた文字列で情報を取得します。

上記の方法で取得した証券コードはint型ですので，文字列型に変換して".T"を後ろにつける必要があります。

In [ ]:
# PrimeCode['雪国まいたけ'] + '.T' ではエラーになる

str(PrimeCode['雪国まいたけ']) + '.T'

'1375.T'